In [ ]:
%load_ext autoreload 
%autoreload 2
%matplotlib notebook
import matplotlib.pyplot as plt
import random
import numpy as np
import sys
import os
os.chdir("/Users/rajpurkar/Documents/Code/ecg")
sys.path.append('./ecg')

## Loading Data

In [ ]:
LOAD_FROM_MODEL = True
LOAD_FROM_FILE = False

model_paths = ['saved/default/1498801603-612/0.430-0.849-013-0.313-0.883.hdf5', 'saved/default/1498801603-783/0.425-0.849-010-0.344-0.871.hdf5', 'saved/default/1498680570-155/0.426-0.848-011-0.339-0.873.hdf5']

import load
import json
import util
import predict

if LOAD_FROM_MODEL is True:
    params = util.get_model_params(model_paths[0])
    test_params = json.load(open('./configs/test.json', 'r'))
    x_gt, gt, processor, dl = load.load_test(
            test_params,
            train_params=params,
            split='test')
    model_probs = predict.get_ensemble_pred_probs(model_paths, x_gt)
elif LOAD_FROM_FILE is True:
    model_probs = None
    #file_to_load = open('./configs/train.json', 'r')
    file_to_load = open('./configs/test.json', 'r')
    params = json.load(file_to_load)
    dl, processor = load.load_train(params)

## Count Patients

In [ ]:
import fnmatch
from collections import defaultdict
from tqdm import tqdm

path = params["data_path"]
ext = '*_grp*.episodes.json'
#ext = '*.episodes.json'

def get_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, ext):
            yield(os.path.join(root, filename))

def patient_id(record):
    return os.path.basename(record).split("_")[0]

class_patients = defaultdict(set)
for f in tqdm(get_files(path)):
    jfile = json.load(open(f, 'r'))
    for episode in jfile['episodes']:
        rhythm_name = episode['rhythm_name']
        if rhythm_name == 'SUDDEN_BRADY':
            rhythm_name = u'CHB'
        if rhythm_name == 'NSR':
            rhythm_name = u'SINUS'
        class_patients[rhythm_name].add(patient_id(f))

for (k, v) in class_patients.items():
    print(k, len(v))

In [ ]:
total = 0
for (k, v) in sorted(class_patients.items()):
    lv = len(v)
    total += lv
    print(k, lv)
print(total)

In [ ]:
x = dl.x_test
y = dl.y_test

def from_one_hot_to_int(label):
    return np.argmax(label, axis=-1)

def get_x_y_predictions_at_index(index, probs=None):
    x_sample = x[index]
    y_sample = from_one_hot_to_int(y[index])
    y_prediction = None
    if probs is not None:
        y_prediction = np.argmax(probs[index], axis=-1)
    return x_sample, y_sample, y_prediction

def get_sample_from_classes(
        categories, min_mistakes = 20, num_tries = 1000, only_classes=False, probs=None):
    classes = np.array([processor.class_to_int[c] for c in categories])
    y_maxed = np.argmax(y, axis=-1)
    indices = np.where(np.array([np.in1d(classes, row).all() for row in y_maxed]))[0]
    for _ in range(num_tries):
        index = random.choice(indices)
        y_prediction = None
        x_sample, y_sample, y_prediction = get_x_y_predictions_at_index(index, probs=probs)
        if only_classes:
            if (set(np.unique(y_sample)) != set(np.unique(classes))):
                continue
        num_wrong = 0
        if y_prediction is None:
            break
        num_wrong = np.sum(y_sample != y_prediction)
        if (num_wrong > min_mistakes):
            print("Prediction got wrong " +  str(num_wrong * 1.0 / len(y_sample)))
            break
    return x_sample, y_sample, y_prediction, index

## Count Distribution

In [ ]:
truths = np.argmax(y, axis=2)
num_outputs_for_thirty_seconds = len(truths[0])
truths_flat = truths.flatten()
classes_unique, counts = np.unique(truths_flat, return_counts=True)
classes_u = np.array(processor.classes)[classes_unique]
num_hours = (counts * 30 / num_outputs_for_thirty_seconds) / 3600.0
for pair in zip(classes_u, num_hours):
    print(pair)

## Visualize Data

In [ ]:
%matplotlib inline
import matplotlib.cm as cm
from itertools import groupby
plt.rcParams["figure.figsize"] = (9, 6)

def from_int_to_name(l):
    return processor.classes[l]

def draw_sample(x_sample, y_sample, y_prediction, step, show_label=True, save=False, small_frame=False):
    colors = cm.Pastel2(np.linspace(0, 1, 20))
    y_times = np.linspace(step/2, len(x_sample) - step/2, len(y_sample))
    if show_label is True:
        grouped_labels = [(k, sum(1 for i in g)) for k,g in groupby(y_sample)]
        acc = 0
        seen = {}
        for label, number in grouped_labels:
            p = {
                "color": colors[label],
                "alpha": 0.5,
                "lw": 0
            }
            if label not in seen:
                label_name = from_int_to_name(label)
                p["label"] = label_name
                seen[label] = True
            plt.axvspan(
                acc * step,
                (acc + number) * step, **p)
            acc += number
    print(np.array(processor.classes)[y_sample])
    if y_prediction is not None:
        print(np.array(processor.classes)[y_prediction])
    plt.plot(x_sample, color='#000000', alpha=1)
    plt.legend(loc="best", prop={'size':14})
    plt.yticks([])
    plt.xticks([])
    plt.tight_layout()
    if small_frame is True:
        plt.xlim([1050, 3050])
    if save is True:
        plt.savefig(str(np.unique(np.array(processor.classes)[y_sample])[0]) + "-" + str(index) + '.pdf', dpi=400, format='pdf',bbox_inches='tight',pad_inches=0)
    plt.show()
    plt.close()

#for class_indiv in processor.classes:
x_sample, y_sample, y_prediction, index = get_sample_from_classes([u'AFIB', u'SINUS'], only_classes=False, probs=model_probs, min_mistakes=0)

#index = 161 # good afib, sinus
#index = 33 # good afib, sinus
#index = 275 # good afib, sinus
index = 169 # good afib, sinus
x_sample, y_sample, y_prediction = get_x_y_predictions_at_index(index)
step = params["step"] if "step" in params else 256
draw_sample(x_sample, y_sample, y_prediction, step, save=True, show_label=False, small_frame=True)


## Evaluating Models

In [ ]:
import evaluate
import human_performance
from tabulate import tabulate

aggregate_data = []
f1_data = []
for metric in ['seq', 'set']:
    # models
    evaluator = evaluate.evaluate_multiclass(
        gt, model_probs, processor.classes, metric, ', '.join(model_paths), display_scores=False)
    model_plotMat, model_support, class_names = evaluate.parse_classification_report(evaluator.scorer.report)
    model_f1 = model_plotMat[:-1, 2]
    f1_data.append(model_f1)
    aggregate_data.append(model_plotMat[-1, :])

    # humans
    test_params_copy = test_params.copy()
    human_ground_truths, human_probs = human_performance.human_gt_and_probs(test_params_copy, x_gt, gt, processor)
    evaluator = evaluate.evaluate_multiclass(
        human_ground_truths, human_probs, processor.classes, metric, ', '.join(model_paths), display_scores=False)
    human_plotMat, human_support, class_names = evaluate.parse_classification_report(evaluator.scorer.report)
    human_f1 = human_plotMat[:-1, 2]
    f1_data.append(human_f1)
    aggregate_data.append(human_plotMat[-1, :])

cell_text = []

f1_data = np.array(f1_data).T
for row, class_name in zip(f1_data, class_names):
    cell_text.append([class_name] + ['%1.3f' % x for x in row])

aggregate_data = np.array(aggregate_data).T
for row, class_name in zip(aggregate_data, ['Precision', 'Recall', 'F1']):
    cell_text.append([class_name] + ['%1.3f' % x for x in row])

table = tabulate(
    cell_text, tablefmt="latex", floatfmt=".3f",
    headers=["Model seq", "Human seq", "Model set", "Human set"])

rows = []
import re
for row in table.split('\n'):
    elems = re.split('\s+', row)
    if len(elems) > 2 and "seq" not in elems:
        for start in [3, 7]:
            end = start+2
            winner = start if float(elems[start]) > float(elems[end]) else end
            elems[winner] = "\\textbf{" + elems[winner] + "}"
    row = " ".join(elems)
    rows.append(row)
table = "\n".join(rows)
print(table)

In [ ]:
2 *binom.cdf(5,19, 0.5)

In [ ]:
import human_performance
from sklearn.metrics import confusion_matrix
from scipy import stats
from scipy.stats import binom
from sklearn import preprocessing
from tabulate import tabulate

lb = preprocessing.MultiLabelBinarizer(range(len(processor.classes)))

def get_binary_preds_from_probs(probs):
    preds = np.argmax(probs, axis=-1)
    multi_label_preds = lb.fit_transform(preds)
    return multi_label_preds

test_params_copy = test_params.copy()
human_probs_all = None
for i in [0,1,2,3,5]:
    test_params_copy["epi_ext"] = "_rev" + str(i) + ".episodes.json"
    _, human_probs, _ = load.load_x_y_with_processor(test_params_copy, processor)
    if human_probs_all is None:
        human_probs_all = human_probs
    else:
        human_probs_all = human_probs + human_probs_all

hb = get_binary_preds_from_probs(human_probs_all)
mb = get_binary_preds_from_probs(model_probs)


p_values = []
for class_int, class_label in enumerate(processor.classes):
    cnf = confusion_matrix(hb[:, class_int], mb[:, class_int])
    if cnf.shape[0] != 2: continue
    y01 = cnf[0, 1]
    y10 = cnf[1, 0]
    McN = (np.abs(y01-y10))**2 / (y10+y01)
    if y01 + y10 >= 25:
        p_value = stats.chi2.sf(McN,1)
    else:
        p_value = 2 * binom.cdf(min(y01, y10), y10 + y01, 0.5)
    # print(class_label, cnf, '%.16f' % p_value)
    p_values.append((p_value, class_label))

alpha = 0.01
n = len(p_values)


cell_text = []

for index, (p_value, class_label) in enumerate(sorted(p_values)):
    threshold = (index + 1) * alpha / n
    reject_null = (p_value <= threshold)
    cell_text.append([class_label, '%.5f' % p_value, reject_null])

    
table = tabulate(
    cell_text, floatfmt=".5f",
    headers=["Class", "P Value", "Reject H0"])

print(table)